In [3]:
import requests
import sqlite3
import configparser
import random
import os

In [4]:
class UptodateApiClient:
    def __init__(self, session_key):
        self.base_url = "https://uptodate.com"
        self.session_key = session_key

    def search_interaction(self, drug_ids):
        if not drug_ids:
            raise ValueError("At least two drug_ids are required for search_interaction")
        
        search_url = f"{self.base_url}/services/app/drug/interaction/search/json?"
        search_url += "&".join([f"drug={drug_id}" for drug_id in drug_ids])
        # print(search_url)
        return self.get_url(search_url)
    
    def get_interaction(self,id):
        if not f"{id}.html" in os.listdir():
            try:
                search_url = f"{self.base_url}/services/app/drug/interaction/{id}/json"
                res = self.get_url(search_url)['data']['interactionHtml']
                self.save_explain(id,res)
                return res
            except:
                print(id)
    
    def autocomplete(self,term,pagesize=9):
        search_url = f"{self.base_url}/services/app/drug/interaction/search/autocomplete/json?"
        search_url += f"term={term}&page=1&pageSize={pagesize}"
        return self.get_url(search_url)

    def get_url(self,search_url):
        cookies = {"utd-PROD-SessionKey": self.session_key}
        response = requests.get(search_url, cookies=cookies)
        self.res = response
        return response.json()

    def save_explain(self,explain_id, explain):
        with open(f'{explain_id}.html','w') as w:
            w.write(explain)
            w.close()

    def fetch_drug_database(self,dbname='drugs.db'):
        all_data=[]
        for ch in "abcdefghigklmnopqrstuvwxyz":
            r = api_client.autocomplete(ch,pagesize=9999)
            if len(r['data']['drugs']) < 9999:
                all_data+=r['data']['drugs']
                print(ch,"-->",str(len(r['data']['drugs'])))
            else: print(ch)
        self.store_drugs(all_data,dbname=dbname)
        return all_data

    def fetch_interaction_database(self,unq, db_name='interactions.db'):
        get_key = lambda inter :",".join(sorted(list(map(lambda x: x['globalId'],inter['items']))))
        interactions = {}
        counter = 0
        lcounter = 0
        stasis=0
        while stasis < 50:
            for sublist in [unq[i:i + 400] for i in range(0, len(unq), 400)]:
                try:
                    r = api_client.search_interaction(sublist)
                    for inter in r['data']['searchResults']:
                        key = get_key(inter)
                        if not key in interactions.keys():
                            interactions[key] = inter
                            counter+=1
                    if counter==lcounter:
                        stasis +=1
                        print(stasis)
                    else:
                        print(str(counter-lcounter)," new interactions Added from totatlly",str(len(r['data']['searchResults'])),"interactions")
                        stasis=0
                    lcounter=counter
                except:
                    pass
            random.shuffle(unq)
        self.store_interactions(interactions,dbname=db_name)
        return interactions

    def store_drugs(self,drugs,dbname='drugs.db'):
        connection = sqlite3.connect(dbname)
        cursor = connection.cursor()

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS drugs (
                id TEXT,
                globalId INTEGER,
                name TEXT
            )
        ''')
        for row in drugs:
            cursor.execute('''
                INSERT INTO drugs (id, globalId, name)
                VALUES (?, ?, ?)
            ''', (row['id'], row['globalId'], row['name']))

        connection.commit()
        connection.close()
    
    def read_drugs(self,debname='drugs.db'):
        connection = sqlite3.connect(debname)
        cursor = connection.cursor()
        cursor.execute('SELECT * FROM drugs')
        rows = cursor.fetchall()
        connection.close()
        drugs = []
        for row in rows:
            drug_dict = {'id': row[0],'globalId':row[1], 'name': row[2]}
            drugs.append(drug_dict)
        return drugs
    
    def store_interactions(self,interactions,dbname='interactions.db'):
        connection = sqlite3.connect(dbname)
        cursor = connection.cursor()

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS interactions (
                id TEXT,
                globalId_1 INTEGER,
                globalId_2 INTEGER,
                name_1 TEXT,
                name_2 TEXT,
                riskRating TEXT,
                url TEXT
            )
        ''')
        for key in interactions.keys():
            cursor.execute('''
                INSERT INTO interactions (id, globalId_1, globalId_2, name_1, name_2, riskRating, url)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (key, interactions[key]['items'][0]['globalId'], interactions[key]['items'][1]['globalId'],
                interactions[key]['items'][0]['name'],interactions[key]['items'][1]['name'],
                interactions[key]['riskRating'],interactions[key]['url']))

        connection.commit()
        connection.close()
    
    def read_interactions(self,dbname='interactions.db'):
        connection = sqlite3.connect(dbname)
        cursor = connection.cursor()

        cursor.execute('''
            SELECT id, globalId_1, globalId_2, name_1, name_2, riskRating, url
            FROM interactions
        ''')
        rows = cursor.fetchall()

        interactions = {}
        for row in rows:
            interactions[row[0]] = {
                'items':[
                    {'globalId':row[1],'name':row[3]},
                    {'globalId':row[2],'name':row[4]}
                ],
                'riskRating': row[5],
                'url': row[6]
            }
        connection.close()
        return interactions

config = configparser.ConfigParser()
config.read('config.ini')
session_key = config.get('API','session_key')
api_client = UptodateApiClient(session_key)

In [5]:
#drugs = api_client.fetch_drug_database()
drugs = api_client.read_drugs(debname='db/drugs.db')
interactions = api_client.read_interactions(dbname='db/interactions.db')

In [6]:
ids = list(map(lambda x: x['id'],drugs))
unq =list(set(ids))

In [7]:
urls = list(map(lambda x: int(interactions[x]['url'].split('/')[-2]),interactions))
unq_urls = list(set(urls))

In [8]:
len(unq_urls)

9967

In [9]:
import multiprocessing
p =multiprocessing.Pool(8)
p.map(api_client.get_interaction,unq_urls)

2705
7087
3530
71553568

2710
63495416

4290
7203
4799
4317
2837
6365
7812
6429
5473
3699
3720
7290
4841
6551
4403
6558
6569
7332
3844
5552
6596
4516
5605
3055
7980
4535
56917986

6663
5719
7500
7501
4570
8069
5786
3107
8079
5154
7509
3991
5159
5793
8083
3122
3993
7512
3123
5795
6760
3131
3997
5167
8092
3142
6772
4004
6776
5179
3160
5809
4613
8126
5810
4616
6789
5188
7604
3168
8130
6796
8132
7625
6799
3184
6808
4036
6810
76338137

5870
4632
4634
6820
3212
7651
8145
6823
4638
5895
8148
4052
5238
5919
8152
5922
4063
3240
6853
5243
5926
5245
7672
5246
6885
3261
5931
3263
81634656

4086
8168
7686
5270
8170
5275
7690
3285
7691
3287
5277
8174
6907
4669
8176
5946
3295
5286
4678
7694
4681
3302
5290
5292
8201
7700
3313
6972
5295
3331
5297
8204
4690
5298
7705
6977
5306
4706
8207
6983
7711
5955
3385
5311
3386
7714
5959
7715
4711
6991
3398
3401
7717
5965
4126
5331
7001
3409
4720
4723
8223
7005
6219
7007
7728
3425
4136
6265
8259
5340
4137
4728
3442
3443
4733
8263
7732
7735
6276
4156
5359
7024
7740
